# Advanced Usage

Here we will introduce some advanced usage of QSPARSE by topics.

## Channel-wise Quantization

Channel-wise quantization denotes the technique to use different decimal bits cross different channels, i.e., quantize each channel independently. It is commonly known that channel-wise quantization can reduce quantization error drastically especially when inter-channel numerical ranges have large variance. By default 

In [1]:
from qsparse import quantize

## Structured Pruning

## Saturated Quantization

## Bias Quantization

## Inspecting Parameters of a Pruned/Quantized Model

## Window Size for Feature Pruning (and Quantization)

## Resuming from Checkpoint 

## Better Logging

## Extending Methods of Quantization And Pruning